In [6]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import os

# Establecer el directorio de trabajo
os.chdir("/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets")

# Leer los archivos CSV en dataframes
df_articulos = pd.read_csv('Articulos.txt', encoding='latin1')
df_proveedores = pd.read_csv('Proveedores.txt', encoding='latin1')
df_merall = pd.read_csv('/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets_Codigo/MerAll.txt', encoding='latin1')

# Combinar dataframes utilizando la columna 'Clapro' como clave en comun
df_stock = pd.merge(df_merall, df_proveedores, how='inner', left_on='Clapro', right_on='Clapro')

# Calcular variables Entrega Habitual & Consumo Promedio
df_stock['Entrega_Habitual'] = (df_stock['Limcrepro'] + df_stock['Clazon']) / 2
df_stock['Consumo_Promedio'] = df_stock['Cant'] / df_stock['Dias_Totales']

In [7]:
# Calcular el Stock Mínimo
df_stock['Stock Minimo'] = df_stock['Entrega_Habitual'] * df_stock['Consumo_Promedio']

# Seleccionar columnas relevantes y renombrar variable Entrega con Retraso
df_stock = df_stock [['Claart', 'Cant', 'Dias_Totales', 'Consumo_Promedio', 'Entrega_Habitual', 'Stock Minimo', 'Clazon']]
df_stock = df_stock.rename(columns={'Clazon': 'Entrega_Retraso'})

In [8]:
# Calcular el Stock Maximo
df_stock['Stock Maximo'] = df_stock['Entrega_Habitual'] * df_stock['Consumo_Promedio'] + df_stock['Stock Minimo']

In [9]:
# Calcular el Stock Mínimo de Seguridad
df_stock['Stock de Seguridad'] = (df_stock['Entrega_Retraso'] - df_stock['Entrega_Habitual']) * df_stock['Consumo_Promedio']
df_stock = df_stock[['Claart', 'Stock Minimo', 'Stock Maximo', 'Stock de Seguridad']]

In [10]:
# Agregar columnas de Existencia y Descripción de Artículo
df_stock = df_stock.merge(df_articulos[['Claart', 'Ex', 'Desart']], on='Claart', how='left')
df_stock = df_stock.rename(columns={'Ex': 'Existencia', 'Desart': 'Descripción'})
df_stock = df_stock [['Claart', 'Descripción', 'Existencia', 'Stock Minimo', 'Stock Maximo', 'Stock de Seguridad']]

In [11]:
# Agregar y calcular Recomendación
df_stock['Recomendacion'] = np.where(df_stock['Existencia'] < df_stock['Stock Minimo'],
                                     df_stock['Stock Minimo'] - df_stock['Existencia'],
                                     np.where(df_stock['Existencia'] > df_stock['Stock Maximo'],
                                              df_stock['Stock Maximo'] - df_stock['Existencia'],
                                              0))

In [7]:
# Guardar dataframe
df_stock.to_csv('/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets_Codigo/Stock.txt', index=False, encoding="latin1")